In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python -m pip install --upgrade pip
!python -m pip install autogluon
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 40.4 MB/s eta 0:00:00
     ━━━━━━

In [42]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
# from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline
# matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin

# PyTorch
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.nn import Parameter
# from torch import Tensor
# from torch.utils.data import DataLoader

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

# Utility
import os
import time
import datetime
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

In [5]:
path = "./drive/MyDrive/Colab Notebooks/data"
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv")
submit = pd.read_csv(path + "/sample_submission.csv")
inter = pd.read_csv(path + "/international_trade.csv")

# rename supply & price
train = train.rename(columns={'supply(kg)':'supply','price(원/kg)':'price'})

In [14]:
# inter

inter['기간']=pd.to_datetime(inter['기간'],format='%Y-%m').apply(lambda x: str(x)[:7])
inter['무역수지']=inter['수출 중량']*inter['수출 금액'] - inter['수입 중량']*inter['수입 금액']

# 타겟 item만 남기기
target_index = []
target_index += list(inter[inter['품목명'].str.contains("감귤")].index)
target_index += list(inter[inter['품목명'].str.contains("브로콜리")].index)
target_index += list(inter[inter['품목명'].str.contains("순무")].index)
target_index += list(inter[inter['품목명'].str.contains("당근")].index)
target_index += list(inter[inter['품목명'].str.contains("양배추")].index)
target_index = list(set(target_index)) # 중복 제거
inter_ = inter.iloc[target_index]

# inter의 품목명 변경
def change_item_name_properly(row):
    item = row['품목명']
    if '감귤' in item:
        item = 'TG'
    elif '브로콜리' in item:
        item='BC'
    elif '무' in item:
        item = 'RD'
    elif '당근' in item:
        item = 'CR'
    elif '양배추' in item :
        item = 'CB'
    row['품목명'] = item
    return row
inter_ = inter_.apply(change_item_name_properly,axis=1)
# 브로콜리의 경우, 복제하여 양배추로 추가해주기
inter_bc=inter_[inter_['품목명']=='BC'].copy()
inter_bc['품목명']='CB'
inter_ = inter_.append(inter_bc,ignore_index=True)

# 기간 -> 'year_month', 품목명 -> 'item' , 무역수지 -> 'trade'
inter_ = inter_.rename(columns={'기간':'year_month','품목명':'item','무역수지':'trade'})
# inter_ = inter_[['year_month','item','trade']]
inter_.head(2)

,year_month,item,수출 중량,수출 금액,수입 중량,수입 금액,trade
0,2022-05,TG,810,9,1,0,7290
1,2019-01,BC,160,1,638913,563,-359707859


In [ ]:
# import plotly.graph_objects as go

# items = ['TG','BC','RD','CR','CB']
# for index,i in enumerate(items):
#   subset = inter_[inter_['item']==i]
#   subset['year_month'] = pd.to_datetime(subset['year_month'],format='%Y-%m')
#   # 수출 중량 그래프
#   fig1 = px.scatter(data_frame=subset,x='year_month',
#                      y='수출 중량',title=f'{i}_export',trendline='ols')

#   fig2 = px.scatter(data_frame=subset,x='year_month',
#                      y='수입 중량',title=f'{i}_import',trendline='ols')
#   fig1.show()
#   fig2.show()


In [17]:
# merge inter with train
train['year_month'] = train['timestamp'].apply(lambda x :str(pd.to_datetime(x).year) + "-" + (str(pd.to_datetime(x).month) if len(str(pd.to_datetime(x).month))>1 else "0"+str(pd.to_datetime(x).month)))
train['trade']=0
for i in inter_.index:
  ym = inter_.loc[i,'year_month']
  item = inter_.loc[i,'item']
  trade = inter_.loc[i,'trade']
  train.loc[(train['item']==item)&(train['year_month']==ym),'trade']=trade
# train = train.drop('trade',axis=1)

In [18]:
train.head(3)

,ID,timestamp,item,corporation,location,supply,price,year_month,id,trade
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019-01,TG_A_J,10039296
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019-01,TG_A_J,10039296
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01,TG_A_J,10039296


In [ ]:


# Scaling price & supply & trade
# price -> np.log1p
# supply -> np.log1p
# trade -> StandardScaler
train_scaled = train.copy()
train_scaled['price'] = np.log1p(train_scaled['price'].values)
train_scaled['supply'] = np.log1p(train_scaled['supply'].values)
train_scaled['trade'] = StandardScaler().fit_transform(np.array(train_scaled['trade'].values).reshape(-1,1))

train_scaled.head()

,ID,timestamp,item,corporation,location,supply,price,year_month,trade
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.000000,0.000000,2019-01,0.423414
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.000000,0.000000,2019-01,0.423414
2,TG_A_J_20190103,2019-01-03,TG,A,J,11.012083,7.455298,2019-01,0.423414
3,TG_A_J_20190104,2019-01-04,TG,A,J,10.126671,7.250636,2019-01,0.423414
4,TG_A_J_20190105,2019-01-05,TG,A,J,10.384462,7.131699,2019-01,0.423414


In [19]:
train['id'] = train['ID'].apply(lambda x:str(x)[:6])
train.head(3)

,ID,timestamp,item,corporation,location,supply,price,year_month,id,trade
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019-01,TG_A_J,10039296
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019-01,TG_A_J,10039296
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019-01,TG_A_J,10039296


In [41]:
len(train)

59397

In [54]:
train_ = train.drop(['ID','item','corporation','location','year_month'],axis=1)
# moving average
df = pd.DataFrame(columns=train_.columns)
for i in train_['id'].unique():
  subset = train_[train_['id']==i]
  org_index = subset.index
  subset.reset_index(inplace=True,drop=True)
  subset_ = subset.iloc[30:,:] #30개
  subset_.reset_index(inplace=True)
  org_index = org_index[30:]
  subset_['ma']=0
  def moving_average(row):
    index = row['index']
    mean = np.mean(subset.loc[index:index+30,'price'].values)
    row['ma'] = mean
    return row
  subset_ = subset_.apply(moving_average,axis=1)
  subset_.index = org_index
  df = pd.concat([df,subset_],axis=0)
df



,timestamp,supply,price,id,trade,index,ma
30,2019-01-31,15997.0,2278.0,TG_A_J,10039296,30.0,1947.225806
31,2019-02-01,36010.0,2349.0,TG_A_J,279642,31.0,1873.741935
32,2019-02-02,8808.0,2286.0,TG_A_J,279642,32.0,1926.741935
33,2019-02-03,0.0,0.0,TG_A_J,279642,33.0,1983.806452
34,2019-02-04,8468.0,1831.0,TG_A_J,279642,34.0,2121.806452
...,...,...,...,...,...,...,...
59392,2023-02-27,452440.0,468.0,RD_F_J,16000,1518.0,525.000000
59393,2023-02-28,421980.0,531.0,RD_F_J,16000,1519.0,539.250000
59394,2023-03-01,382980.0,574.0,RD_F_J,0,1520.0,542.000000
59395,2023-03-02,477220.0,523.0,RD_F_J,0,1521.0,526.000000


In [55]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df.drop(columns=['supply'],axis=1),
    id_column='id',
    timestamp_column = 'timestamp'
)

In [56]:
predictor = TimeSeriesPredictor(prediction_length=28,
                                path = './autogluon_jeju',
                                target='price',
                                eval_metric='RMSE').fit(train_data,
                                presets="medium_quality")

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: medium_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'medium_quality',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': None,
 'target': 'price',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 58227 rows, 39 items (item = single time series). Average time series length is 1493.0. Data frequency is 'D'.
AutoGluon will save models to ./autogluon_jeju/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price'
	past covariates:  ['trade', 'index', 'ma']

Starting training. Start time is 2

In [57]:
preds = predictor.predict(train_data)
preds

INFO:lightning_fabric.utilities.seed:Global seed set to 123
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3136.449973  1096.640961  1790.025673  2283.624321   
        2023-03-05   619.149688 -1825.980844  -956.694892  -354.391237   
        2023-03-06  3028.739395  -138.826746   983.151641  1729.370910   
        2023-03-07  3398.616455   -10.518765  1182.793197  1999.069013   
        2023-03-08  3290.578759  -356.755408   861.414776  1796.032131   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   434.947754  -209.494095   -36.132169   117.450651   
        2023-03-28   473.136745  -203.681397   -25.441359   145.210431   
        2023-03-29   466.511601  -208.634257   -28.370306   135.775688   
        2023-03-30   449.151086  -227.361928   -42.896166    88.930820   
        2023-03-31   422.662177  -253.036085   -59.823479    74.413835   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2737.825168  3147.546558  3554.166090  3992.032823   
        2023-03-05   146.090550   604.828268  1085.903709  1599.946882   
        2023-03-06  2381.528084  2965.266094  3544.980625  4218.963819   
        2023-03-07  2708.822952  3359.961453  4026.848356  4735.466699   
        2023-03-08  2568.255882  3274.770892  3981.677441  4761.693799   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   305.379500   445.235118   569.213147   707.930888   
        2023-03-28   327.350912   479.385533   620.185005   758.421116   
        2023-03-29   338.807669   487.334891   613.414434   755.599266   
        2023-03-30   279.751794   443.704228   595.168018   751.465846   
        2023-03-31   256.197814   410.759957   568.407028   730.953952   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4465.151545  5228.313189  
        2023-03-05  2211.251730  3081.497917  
        2023-03-06  4953.431259  6037.672931  
        2023-03-07  5569.689792  6785.206147  
        2023-03-08  5698.182767  7000.452294  
...                         ...          ...  
RD_F_J  2023-03-27   857.414686  1063.234042  
        2023-03-28   907.684341  1180.929484  
        2023-03-29   916.723434  1144.623622  
        2023-03-30   918.193918  1157.384690  
        2023-03-31   896.006039  1132.924280  

[1092 rows x 10 columns]

In [58]:
preds['mean'].apply(lambda x : 0 if x<0 else x).describe()

count    1092.000000
mean     1444.198516
std      1430.476900
min         0.000000
25%       197.637232
50%       659.777808
75%      2685.720302
max      4916.796898
Name: mean, dtype: float64

In [59]:
submit.loc[:,'answer'] = preds['mean'].apply(lambda x : 0 if x<0 else x).values
submit

,ID,answer
0,TG_A_J_20230304,3136.449973
1,TG_A_J_20230305,619.149688
2,TG_A_J_20230306,3028.739395
3,TG_A_J_20230307,3398.616455
4,TG_A_J_20230308,3290.578759
...,...,...
1087,RD_F_J_20230327,434.947754
1088,RD_F_J_20230328,473.136745
1089,RD_F_J_20230329,466.511601
1090,RD_F_J_20230330,449.151086


In [60]:
submit.set_index('ID').to_csv(path+'/autogluon_moving_average30_trade.csv')

---

### ↓↓↓↓ Check results ↓↓↓↓

In [61]:
# Check Results

# Basic Model prediction
# basic = pd.read_csv(path+'/darts_submit_basic.csv',
#                     index_col=0).rename(columns={'answer':'basic'})
add_trade_by_yearmonth = pd.read_csv(path+'/darts_add_trade_by_yearmonth.csv',
                    index_col=0).rename(columns={'answer':'trade_by_yearmonth'})
supply_trade_scaling = pd.read_csv(path+'/darts_supply_trade_scaling.csv',
                    index_col=0).rename(columns={'answer':'supply_trade_scaling'})
trade_scaling = pd.read_csv(path+'/darts_trade_scaling.csv',
                            index_col=0).rename(columns={'answer':'trade_scaling'})
ts_inputchunck90 = pd.read_csv(path+'/darts_ts_inputchunck90.csv',
                            index_col=0).rename(columns={'answer':'ts_inputchunck90'})
atg_ma_trade = pd.read_csv(path+'/autogluon_moving_average_trade.csv',
                           index_col=0).rename(columns={'answer':'atg_ma_trade'})
atg_ma30_trade = pd.read_csv(path+'/autogluon_moving_average30_trade.csv',
                           index_col=0).rename(columns={'answer':'atg_ma30_trade'})

# prediction concatenated
concat_submits = pd.concat([
                            add_trade_by_yearmonth,
                            supply_trade_scaling,
                            trade_scaling,
                            ts_inputchunck90,
                            atg_ma_trade,
                            atg_ma30_trade],
                           axis=1)
concat_submits

,trade_by_yearmonth,supply_trade_scaling,trade_scaling,ts_inputchunck90,atg_ma_trade,atg_ma30_trade
ID,,,,,,
TG_A_J_20230304,2250.972427,2081.732330,2079.071545,2077.166924,3235.959646,3136.449973
TG_A_J_20230305,1180.577455,2.290948,3.207200,3.081535,705.225924,619.149688
TG_A_J_20230306,2237.264121,1214.577388,2786.772753,1129.879733,3103.308288,3028.739395
TG_A_J_20230307,2470.721502,1785.221528,1551.995756,1491.905602,3448.272293,3398.616455
TG_A_J_20230308,2280.571975,1362.519734,1521.890965,796.456409,3259.190978,3290.578759
...,...,...,...,...,...,...
RD_F_J_20230327,395.836530,160.442993,153.935251,168.241589,496.844670,434.947754
RD_F_J_20230328,379.605563,197.580670,148.903375,211.580329,541.431539,473.136745
RD_F_J_20230329,389.416720,208.351817,202.948779,146.102991,546.648669,466.511601
